# ARIMA / SARIMA / SARIMAX：训练与部署安排（速查）
_更新时间：2026-01-29 04:35_

> 目标：在 dataset interview 的时间约束下，用一套可解释、可复现、可落盘的流程完成 **ARIMA/SARIMA（含外生变量 SARIMAX）** 的训练、验证、预测与交付。  
> 口径：严格 walk-forward；任何随机切分都视为无效。


## 0. 环境与依赖（开局 3–5 分钟）

- Python / Jupyter 可用
- 依赖优先级：
  1. `statsmodels`（ARIMA/SARIMAX 主力）
  2. `pmdarima`（auto_arima，可选）
  3. 只用 `numpy/pandas` 的 fallback（只能做基线与分解，不做完整 ARIMA）

**固定约定**
- `SEED = 42`
- `time_col = ...`
- `target_col = ...`
- `group_col = ...`（可空）
- 预测地平线：`H = ...`
- 季节周期：`m = ...`（典型：24/48/96/168/12 等）


In [ ]:
import os
import json
import math
import time
import warnings
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
import joblib

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 200)
SEED = 42
np.random.seed(SEED)

# statsmodels（若不可用，后续代码会提示并走简化流程）
try:
    import statsmodels.api as sm
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    from statsmodels.tsa.stattools import adfuller, acf, pacf
    STATS_MODELS_OK = True
except Exception as e:
    STATS_MODELS_OK = False
    print("statsmodels not available:", repr(e))


## 1. 数据准备（10–20 分钟）

### 1.1 时间索引与频率
- 时间列转 `datetime`
- 排序：`[group_col, time_col]` 或 `time_col`
- 频率确认：是否等间隔；缺口如何处理（补齐/插值/保持缺失）

### 1.2 目标清洗
- 异常值：clip / winsorize（只在训练期拟合阈值）
- 变换：`log1p`（若严格非负且长尾）
- 缺失：
  - 训练：保留缺失并交给状态空间（可行时）
  - 简化：短缺口线性插值；长缺口切段

### 1.3 外生变量（SARIMAX）
- 所有 exog 必须在预测时点可得
- exog 的滞后与滚动：严格只用过去


In [ ]:
# df = pd.read_csv("train.csv")
# df[time_col] = pd.to_datetime(df[time_col])

# if group_col in df.columns:
#     df = df.sort_values([group_col, time_col]).reset_index(drop=True)
# else:
#     df = df.sort_values([time_col]).reset_index(drop=True)

# df.head()


## 2. 快速诊断（5–15 分钟）

### 2.1 目标是否需要差分（d）
- 画图：level/trend
- ADF（可用时）：p-value 过大 → 非平稳 → 差分
- 经验：先试 `d ∈ {0,1}`，尽量不做高阶差分

### 2.2 是否存在季节性（m）
- 常见 m：
  - 小时级：24（按天）/168（按周）
  - 15min：96（按天）/672（按周）
  - 月度：12（按年）
- 直观判断：按周期叠图、按周期均值

### 2.3 ACF/PACF（可用时）
- ACF 拖尾 / 截尾 → MA 候选 q
- PACF 截尾 → AR 候选 p
- 季节峰值（在 m、2m、...）提示季节项 P/Q


In [ ]:
def try_adf(y: pd.Series):
    if not STATS_MODELS_OK:
        return None
    y = y.dropna().astype(float)
    if len(y) < 20:
        return None
    stat, pvalue, *_ = adfuller(y, autolag="AIC")
    return {"adf_stat": float(stat), "pvalue": float(pvalue), "n": int(len(y))}

def try_acf_pacf(y: pd.Series, nlags: int = 48):
    if not STATS_MODELS_OK:
        return None
    y = y.dropna().astype(float)
    if len(y) < nlags + 5:
        return None
    return {
        "acf": acf(y, nlags=nlags, fft=True).tolist(),
        "pacf": pacf(y, nlags=nlags, method="ywm").tolist(),
    }

# 示例：单序列
# y = df.set_index(time_col)[target_col]
# try_adf(y)


## 3. Walk-forward 验证（核心）

### 3.1 术语
- 训练窗口：过去 N（expanding 或固定长度 rolling）
- 验证窗口：未来 H
- 每个 fold：只用训练窗口拟合 → 预测验证窗口 → 记录指标

### 3.2 两种常用设置
- Expanding（更贴近真实不断累积数据）
- Rolling（更贴近只关心最近分布）

### 3.3 指标
- 回归：MAE/RMSE/SMAPE
- 区间：coverage（若输出置信区间）


In [ ]:
def rmse(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def smape(y_true, y_pred, eps=1e-9):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    denom = np.maximum(eps, (np.abs(y_true) + np.abs(y_pred)) / 2.0)
    return float(np.mean(np.abs(y_pred - y_true) / denom))

def walk_forward_splits(n: int, n_splits: int = 5, min_train_frac: float = 0.6):
    start_train = int(n * min_train_frac)
    step = (n - start_train) // n_splits
    splits = []
    for k in range(n_splits):
        train_end = start_train + k * step
        val_end = min(n, train_end + step)
        tr = np.arange(0, train_end)
        va = np.arange(train_end, val_end)
        if len(va) == 0:
            break
        splits.append((tr, va))
    return splits


## 4. 模型训练（ARIMA / SARIMA / SARIMAX）

### 4.1 形式
- ARIMA(p, d, q)
- SARIMA(p, d, q) × (P, D, Q, m)
- SARIMAX：在上述基础上加入 exog

### 4.2 训练顺序（时间受限）
1. 朴素基线（lag-1 / seasonal naive）
2. 小网格搜索（p,q ∈ {0,1,2}；P,Q ∈ {0,1}；d,D ∈ {0,1}）
3. 用 AIC/BIC + walk-forward 指标双重筛选
4. 最终只保留 1–2 个候选，写清楚理由

### 4.3 常见稳定性设置
- `enforce_stationarity=False`
- `enforce_invertibility=False`
- `simple_differencing=True`（必要时）


In [ ]:
from dataclasses import dataclass
from typing import Tuple, Optional
import numpy as np
import pandas as pd

@dataclass
class SarimaOrder:
    order: Tuple[int, int, int]               # (p,d,q)
    seasonal_order: Tuple[int, int, int, int] # (P,D,Q,m)
    trend: str = "n"                          # "n","c","t","ct"

def seasonal_naive_forecast(y: np.ndarray, H: int, m: int):
    if m is None or m <= 0 or len(y) <= m:
        last = y[-1]
        return np.repeat(last, H)
    base = y[-m:]
    reps = int(np.ceil(H / m))
    return np.tile(base, reps)[:H]

def fit_sarimax(y_train: pd.Series, exog_train: Optional[pd.DataFrame], spec: SarimaOrder):
    if not STATS_MODELS_OK:
        raise RuntimeError("statsmodels not available")
    model = SARIMAX(
        y_train.astype(float),
        exog=exog_train,
        order=spec.order,
        seasonal_order=spec.seasonal_order,
        trend=spec.trend,
        enforce_stationarity=False,
        enforce_invertibility=False,
    )
    res = model.fit(disp=False)
    return res

def forecast_sarimax(res, steps: int, exog_future: Optional[pd.DataFrame]):
    pred = res.get_forecast(steps=steps, exog=exog_future)
    mean = pred.predicted_mean.values
    ci = pred.conf_int(alpha=0.05)
    return mean, ci


## 5. 小网格搜索（SARIMA）

### 5.1 搜索空间（面试版）
- 非季节：p,q ∈ {0,1,2}；d ∈ {0,1}
- 季节：P,Q ∈ {0,1}；D ∈ {0,1}
- 周期：m 由数据频率确定（固定）

### 5.2 选择准则
- 先看 AIC（快速过滤）
- 再看 walk-forward MAE/RMSE（真实效果）
- 再看残差：是否还有明显季节性/自相关


In [ ]:
from itertools import product
import pandas as pd

def candidate_specs(m: int,
                    p_set=(0,1,2), d_set=(0,1), q_set=(0,1,2),
                    P_set=(0,1), D_set=(0,1), Q_set=(0,1),
                    trend_set=("n","c")):
    specs = []
    for p,d,q,P,D,Q,tr in product(p_set, d_set, q_set, P_set, D_set, Q_set, trend_set):
        specs.append(SarimaOrder(order=(p,d,q), seasonal_order=(P,D,Q,m), trend=tr))
    return specs

def quick_aic_rank(y: pd.Series, exog: Optional[pd.DataFrame], specs: List[SarimaOrder], max_try: int = 30):
    rows = []
    tried = 0
    for spec in specs:
        if tried >= max_try:
            break
        try:
            res = fit_sarimax(y, exog, spec)
            rows.append({
                "order": str(spec.order),
                "seasonal_order": str(spec.seasonal_order),
                "trend": spec.trend,
                "aic": float(res.aic),
                "bic": float(res.bic),
            })
            tried += 1
        except Exception:
            continue
    return pd.DataFrame(rows).sort_values("aic")


## 6. Walk-forward 评估（最终以这个表为准）

### 6.1 评估粒度
- 单序列：直接按时间滚动
- 多序列（group）：
  - 逐组训练（慢但干净）
  - 或对每组复用同一组超参（快，面试常用）

### 6.2 记录
- 每 fold：训练区间、验证区间、参数、AIC、MAE/RMSE/SMAPE
- 汇总：平均与标准差


In [ ]:
import numpy as np
import pandas as pd
from typing import Optional

def walk_forward_eval_sarima(y: pd.Series, exog: Optional[pd.DataFrame],
                           spec: SarimaOrder, n_splits: int = 5, min_train_frac: float = 0.6):
    n = len(y)
    splits = walk_forward_splits(n, n_splits=n_splits, min_train_frac=min_train_frac)
    rows = []
    oof = np.full(n, np.nan)

    for k, (tr, va) in enumerate(splits, 1):
        y_tr = y.iloc[tr]
        y_va = y.iloc[va]

        ex_tr = exog.iloc[tr] if exog is not None else None
        ex_va = exog.iloc[va] if exog is not None else None

        steps = len(va)

        try:
            res = fit_sarimax(y_tr, ex_tr, spec)
            pred_mean, _ = forecast_sarimax(res, steps=steps, exog_future=ex_va)
        except Exception:
            pred_mean = seasonal_naive_forecast(y_tr.values, steps, m=spec.seasonal_order[-1])

        oof[va] = pred_mean

        rows.append({
            "fold": k,
            "n_train": int(len(tr)),
            "n_val": int(len(va)),
            "order": str(spec.order),
            "seasonal_order": str(spec.seasonal_order),
            "trend": spec.trend,
            "MAE": float(np.mean(np.abs(y_va.values - pred_mean))),
            "RMSE": rmse(y_va.values, pred_mean),
            "SMAPE": smape(y_va.values, pred_mean),
        })

    return pd.DataFrame(rows), oof


## 7. 最终训练与落盘（交付物）

### 7.1 最终训练
- 用全部训练期数据拟合一次最终模型
- 若存在 holdout（最后一段时间）：只在最后汇报，训练仍用更早数据

### 7.2 落盘文件
- `model.joblib`：statsmodels 结果对象（或只保存参数 + 重建逻辑）
- `spec.json`：(p,d,q)(P,D,Q,m) + trend + 变换信息
- `metrics.csv`：walk-forward 指标
- `oof_predictions.csv`：时间戳、真实值、OOF 预测
- `forecast_example.csv`：未来 H 步预测样例（含置信区间）


In [ ]:
def save_artifacts(output_dir: str,
                   spec: SarimaOrder,
                   metrics_df: pd.DataFrame,
                   oof: np.ndarray,
                   df_base: pd.DataFrame,
                   time_col: str,
                   target_col: str,
                   model_res=None,
                   extra: Optional[Dict]=None):
    os.makedirs(output_dir, exist_ok=True)

    spec_dict = {
        "order": spec.order,
        "seasonal_order": spec.seasonal_order,
        "trend": spec.trend,
        "extra": extra or {},
    }
    with open(os.path.join(output_dir, "spec.json"), "w", encoding="utf-8") as f:
        json.dump(spec_dict, f, ensure_ascii=False, indent=2)

    metrics_df.to_csv(os.path.join(output_dir, "metrics.csv"), index=False)

    out = df_base[[time_col]].copy()
    out["y_true"] = df_base[target_col].values
    out["y_pred"] = oof
    out.to_csv(os.path.join(output_dir, "oof_predictions.csv"), index=False)

    if model_res is not None:
        joblib.dump(model_res, os.path.join(output_dir, "model.joblib"))

    return output_dir


## 8. 部署口径（面试版：最小可运行）

### 8.1 Batch scoring（最常见）
输入：历史到当前时点的序列（以及未来 exog）  
输出：未来 H 步预测（mean + CI）

- 预测从最后一个已知时间点接上
- SARIMAX：未来 exog 必须提供；否则只能输出到 exog 已知的最远处

### 8.2 Online / Streaming（表达方式）
- 每个 group 维护一个状态：最近数据窗口 + 已拟合模型/参数
- 新数据到来：append →（可选）增量更新/定期重训 → forecast

### 8.3 模型更新节奏（执行计划）
- 固定周期重训：每日/每周（rolling window 长度固定）
- 触发式重训：
  - 残差均值漂移
  - 残差方差上升（波动 regime 变化）
  - coverage 明显偏离目标（若输出区间）


In [ ]:
def batch_forecast_from_disk(model_path: str,
                            spec_path: str,
                            history: pd.Series,
                            H: int,
                            exog_future: Optional[pd.DataFrame] = None):
    with open(spec_path, "r", encoding="utf-8") as f:
        spec_dict = json.load(f)
    spec = SarimaOrder(tuple(spec_dict["order"]), tuple(spec_dict["seasonal_order"]), spec_dict.get("trend","n"))

    if STATS_MODELS_OK and os.path.exists(model_path):
        res = joblib.load(model_path)
        mean, ci = forecast_sarimax(res, steps=H, exog_future=exog_future)
        return mean, ci

    mean = seasonal_naive_forecast(history.values, H, m=spec.seasonal_order[-1])
    ci = None
    return mean, ci


## 9. 监控与回归测试（最小三件事）

### 9.1 数据质量
- 目标缺失率、极端值比例
- 时间间隔异常（跳点/重复）
- exog 缺失率（SARIMAX 必做）

### 9.2 预测质量
- 残差均值（bias）
- 残差方差（vol）
- 指标随时间滚动（MAE/RMSE/SMAPE）

### 9.3 模型假设检查（简版）
- 残差 ACF 是否仍有显著峰值
- 季节峰值是否重新出现


In [ ]:
def residual_diagnostics(y_true: np.ndarray, y_pred: np.ndarray, m: Optional[int] = None):
    r = y_true - y_pred
    out = {
        "res_mean": float(np.nanmean(r)),
        "res_std": float(np.nanstd(r)),
        "res_mae": float(np.nanmean(np.abs(r))),
    }
    if m and m > 1 and len(r) > m:
        out["seasonal_res_mean_abs"] = float(np.nanmean(np.abs(r[-m:])))
    return out


## 10. 常见失败模式与应对口径

- 训练不收敛 / 参数爆炸：
  - 降低阶数（先到 (1,1,1)×(0,1,1,m)）
  - 关闭 stationarity/invertibility 强制
  - 缩短训练窗口（rolling）
- 过差分（d/D 太大）：
  - 回退到 d=1、D=1 的最小组合
- 季节周期设错：
  - 残差 ACF 在 m 附近反复出现峰值
  - 更换 m 或加入季节项（P/Q）
- SARIMAX 外生变量不可得：
  - 预测地平线只能到 exog 已知的最远处
  - 回退到纯 SARIMA


## 11. 最终检查清单（提交前 3 分钟）

- [ ] 时间排序正确；频率合理
- [ ] m 的定义写清楚（与数据频率对应）
- [ ] walk-forward 指标表已生成并保存
- [ ] spec.json 已保存（order/seasonal_order/trend/变换）
- [ ] model.joblib 已保存（若 statsmodels 可用）
- [ ] 未来 H 步预测样例输出（含 CI 或说明 fallback）
- [ ] PPT 里明确：选阶数理由、验证方式、失败模式与后续计划
